# Échantillonnage et théorème de Shannon.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, VBox, HBox, Layout
from IPython.display import display

# --- Paramètres de l'Affichage Spectral Fixe ---
F_LIM_MIN = -50.0
F_LIM_MAX = 50.0

# --- Fonction principale d'Analyse et de Plot ---
def visualize_aliasing(F_signal, F_s, T_total, N_fft_points):
    """
    Génère le signal sinusoïdal, l'échantillonne et affiche les domaines
    temporel et spectral, avec une échelle de fréquence fixe [-50 Hz, 50 Hz].
    """
    
    # Validation minimale
    if F_s <= 0 or F_signal <= 0 or N_fft_points <= 0 or T_total <= 0:
        print("Les paramètres doivent être positifs.")
        return

    # --- Paramètres calculés ---
    N_sampled = N_fft_points
    T_sample = N_sampled / F_s          # Durée totale d'échantillonnage
    dt_sample = 1 / F_s                 # Pas de temps d'échantillonnage
    F_nyquist = F_s / 2                 # Fréquence de Nyquist

    # --- Préparation des signaux ---
    
    # Signal continu (pour la référence)
    F_s_high = 100 * F_s if F_s > 0 else 1000 
    N_high = int(F_s_high * T_sample)
    t_high = np.linspace(0, T_sample, N_high, endpoint=False)
    signal_analog = np.sin(2 * np.pi * F_signal * t_high)
    
    # Signal échantillonné
    t_sampled = np.linspace(0, T_sample, N_sampled, endpoint=False)
    signal_sampled = np.sin(2 * np.pi * F_signal * t_sampled)

    # --- 1. Domaine Temporel ---
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    ax1.plot(t_high, signal_analog, label='Signal Analogique $x(t)$', color='blue', linestyle='-')
    
    # Utilisation de plt.stem pour le signal échantillonné
    ax1.stem(
        t_sampled, 
        signal_sampled, 
        linefmt='r-',      # Tiges rouges
        markerfmt='ro',    # Marqueurs ronds rouges
        basefmt=' ',       # Supprime la ligne de base
        label=f'Échantillons $x[n]$'
    )
    ax1.plot(t_sampled, signal_sampled, 'r--', alpha=0.9) 

    
    ax1.set_title(f"Domaine Temporel | $F_0$={F_signal:.1f} Hz, $F_e$={F_s:.1f} Hz")
    ax1.set_xlabel('Temps (s)')
    ax1.set_ylabel('Amplitude')
    ax1.set_xlim(0, T_total) # Zoom
    ax1.grid(True)
    ax1.legend(loc='upper right')
    
    # Affichage de l'aliasing
    F_alias_val = 0.0
    if F_signal > F_nyquist:
        k_alias = np.round(F_signal / F_s)
        F_alias_val = np.abs(F_signal - k_alias * F_s)
        if F_alias_val > F_nyquist: 
            F_alias_val = F_s - F_alias_val
        
        ax1.text(0.005, 0.936, 
                 f"⚠️ ALIASING: $F_0 > F_e/2$. Fréquence apparente: {F_alias_val:.2f} Hz", 
                 transform=ax1.transAxes, 
                 color='red', 
                 fontsize=12, 
                 bbox=dict(facecolor='yellow', alpha=0.5))
    
    
    # --- 2. Domaine Spectral (Périodisation) ---
    
    # Calcul de la TFD du signal échantillonné (pour le TRACÉ ROUGE)
    spectrum = np.fft.fft(signal_sampled)
    magnitude_spectrum = np.abs(spectrum) / N_sampled
    frequencies = np.fft.fftfreq(N_sampled, d=dt_sample)
    frequencies_centered = np.fft.fftshift(frequencies)
    magnitude_centered = np.fft.fftshift(magnitude_spectrum)
    
    # CRÉATION DU SPECTRE ANALOGIQUE IDÉAL (pour le TRACÉ BLEU)
    N_analog = N_sampled * 4 
    F_s_analog = 1000 # Très haute Fe
    dt_analog = 1 / F_s_analog
    t_analog = np.linspace(0, N_analog / F_s_analog, N_analog, endpoint=False)
    signal_analog_fft = np.sin(2 * np.pi * F_signal * t_analog)
    
    spectrum_ideal = np.abs(np.fft.fft(signal_analog_fft)) / N_analog
    frequencies_ideal = np.fft.fftfreq(N_analog, d=dt_analog)
    
    frequencies_analog_ideal = np.fft.fftshift(frequencies_ideal)
    magnitude_analog_ideal = np.fft.fftshift(spectrum_ideal)
    
    # --- Tracé du spectre périodisé ---
    
    ax2.set_title(rf'Domaine Spectral (Spectre Périodisé)')
    
    # Plage étendue pour les copies
    MAX_SHIFT = int(np.ceil(F_LIM_MAX / F_s)) + 1 if F_s > 0 else 2 
    labels_drawn = set()
    
    # --- TRACÉ 1 : Copies du Spectre ÉCHANTILLONNÉ (ROUGE, PÉRIODIQUE) ---
    # Label unique pour tous les tracés rouges
    sampled_label = r'Spectre Échantillonné Périodisé $\left|X_{e}(f)\right|$ ($\pm k F_e$)'

    for k in range(-MAX_SHIFT, MAX_SHIFT + 1):
        
        is_principal = (k == 0)
        color = 'red'
        linestyle = '-' if is_principal else '--'
        alpha = 0.6 if is_principal else 0.3 
        
        x_plot = frequencies_centered + k * F_s 
        y_plot = magnitude_centered
        
        # Attribution du label UNIQUEMENT à la première copie tracée (k=0)
        label = sampled_label if k == 0 else None
        if label:
            # S'assurer qu'on ne l'ajoute qu'une seule fois
            if sampled_label in labels_drawn:
                label = None
            else:
                labels_drawn.add(sampled_label)
        
        ax2.plot(
            x_plot, 
            y_plot, 
            color=color, 
            linestyle=linestyle,
            alpha=alpha,
            label=label
        )

    # --- TRACÉ 2 : Spectre ANALOGIQUE IDÉAL (BLEU, TRACÉ EN DERNIER) ---
    analog_label = r'Spectre Analogique Idéal $\left|X(f)\right|$ (Référence, $\pm F_0$)'
    ax2.plot(
        frequencies_analog_ideal, 
        magnitude_analog_ideal, 
        color='blue', 
        linestyle='-',
        alpha=1.0, 
        label=analog_label
    )
    labels_drawn.add(analog_label)
    
    # --- Lignes de Référence ---
    # Fréquence de Nyquist (limite de la bande principale)
    nyquist_label = rf'Fréquence de Nyquist $\pm F_e/2$ ({F_nyquist:.1f} Hz)'
    ax2.axvline(F_nyquist, color='green', linestyle='--', label=nyquist_label if nyquist_label not in labels_drawn else None)
    ax2.axvline(-F_nyquist, color='green', linestyle='--')
    labels_drawn.add(nyquist_label)
    
    # Fréquence d'échantillonnage (centre des copies)
    fs_label = rf'Fréquence d\'échantillonnage $\pm F_e$'
    ax2.axvline(F_s, color='gray', linestyle=':', label=fs_label if fs_label not in labels_drawn else None)
    ax2.axvline(-F_s, color='gray', linestyle=':')
    labels_drawn.add(fs_label)
    
    # --- Mise en évidence de l'Aliasing ---
    if F_signal > F_nyquist:
        # Ligne de la fréquence repliée (le pic rouge observé dans la bande principale)
        alias_line_label = rf'$F_{{alias}}$ Repliée ({F_alias_val:.1f} Hz)'
        ax2.axvline(F_alias_val, color='red', linestyle='-.', linewidth=1.5, alpha=0.9, label=alias_line_label if alias_line_label not in labels_drawn else None)
        ax2.axvline(-F_alias_val, color='red', linestyle='-.', linewidth=1.5, alpha=0.9)
        labels_drawn.add(alias_line_label)

    ax2.set_xlabel('Fréquence (Hz)')
    ax2.set_ylabel('Magnitude')
    
    # FIXER LA LIMITE À [-50 Hz, 50 Hz]
    ax2.set_xlim(F_LIM_MIN, F_LIM_MAX) 
    ax2.set_ylim(0, 0.55) 
    ax2.grid(True)
    
    # Affichage de la légende en bas à droite
    ax2.legend(loc='lower right', fontsize='small')
    
    plt.tight_layout()
    plt.show()

# --- Configuration des Widgets ---

# 1. Fréquence du Signal F0
slider_f_signal = FloatSlider(value=10.0, min=1.0, max=50.0, step=0.5, description=r'F0 (Hz):', layout=Layout(width='300px'))

# 2. Fréquence d'Échantillonnage Fe
slider_f_s = FloatSlider(value=25.0, min=5.0, max=60.0, step=0.5, description=r'Fe (Hz):', layout=Layout(width='300px'))

# 3. Durée d'Affichage Temporelle T_total (Zoom sur le temps)
slider_t_total = FloatSlider(value=2, min=0.1, max=5.0, step=0.1, description=r'Durée (s):', layout=Layout(width='300px'))

# Organisation des widgets
controls_time = HBox([slider_f_signal, slider_f_s])
controls_plot = HBox([slider_t_total])
widget_layout = VBox([controls_time, controls_plot])

# Création de l'interface interactive
interactive_plot = interactive(
    visualize_aliasing, 
    F_signal=slider_f_signal, 
    F_s=slider_f_s,
    T_total=slider_t_total,
    N_fft_points=8192 # Valeur fixée pour la finesse du tracé
)

#### ⚙️ Contrôles Interactifs du Signal et de l'Analyse Spectrale ⚙️

In [11]:
# Affichage
display(widget_layout)
print(f"L'échelle spectrale est fixe entre {F_LIM_MIN} Hz et {F_LIM_MAX} Hz.")
interactive_plot.update()
display(interactive_plot.children[-1])


L'échelle spectrale est fixe entre -50.0 Hz et 50.0 Hz.


Output()

### Tracé supérieur : signaux en fonction du temps
- Le signal analogique $x(t)$ est un signal sinusoidal de fréquence $F_0$ tel que $x(t) = \sin(2\pi F_0 t)$. Ce signal est représenté en fonction du temps $t$ (exprimé en $s$) en bleu.

- Le signal $x(t)$ est échantillonné à une fréquence $F_e$ pour produire le signal échantillonné $x_e(t) = \sin(2\pi F_0 nT_e) \delta(t-nT_e)$ avec $T_e = 1/F_e$ la période d'échantillonnage. Ce signal est représenté en rouge. Chaque échantillon est artificiellement relié à l'échantillon suivant par une ligne en pointillé afin de mieux visualiser l'effet du choix de $F_e$.

### Tracé inférieur : module de la transformée de Fourier des signaux
- Le signal analogique bleu est un sinus pur : son spectre $X(f)$ est donc constitué uniquement de 2 fréquences $F_0$ et $-F_0$ avec une amplitude de 1/2. Cette allure n'est bien sûr modifiée que par le choix de la valeur de $F_0$ ;

- Le signal échantillonné possède le même spectre que le signal analogique, mais périodisé tous les multiples de $F_e$, de sorte que $X_e(f) = \sum_k X(f-kF_e)$.

### Mise en évidence du théorème de Shannon :
Dès que la fréquence $F_e$ est choisie inférieure à $2F_0$, on voit clairement :
- dans le domaine temporel, les échantillons du signal $x_e(t)$ semblent dessiner un nouveau signal sinusoidal, de fréquence $F<F_0$
- dans le domaine fréquentiel, les raies issues de la périodisation du motif spectral de $X(f)$ empiètent sur le spectre d'origine
Dès lors, on ne sait plus attribuer les échantillons au signal $x(t)$ d'origine de fréquence $F_0$ ou au signal **replié** de fréquence $F<F_0$



*v1.0, 1/12/2025, Sylvain ARGENTIERI*